In [55]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import re
from random import uniform
import time
from fake_headers import Headers

In [56]:
def get_soup(url):
    try:
        response = requests.get(url)
        response.raise_for_status() 
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    except requests.RequestException:
        return False

In [62]:
def get_urls():
    links = []
    for i in range(70):
        url = f'https://www.film.ru/a-z/movies/nojs?page={i}'
        soup = get_soup(url)
        data = soup.find_all(class_ = 'film_list_link')
        all_links = [i['href'] for i in data]
        links.append(all_links)
        links_flat = [n for k in links for n in k]
    return links_flat

In [63]:
def parse_web():
    list_links = get_urls()
    final=[]
    for l in list_links: 
        home_page = 'https://www.film.ru'
        li = home_page + str(l)
        soup_1 = get_soup(li)
        info = soup_1.find(class_ = "wrapper_movies_top_main_right")
        info = str(info('h1')).split("<br/>")
        info_1 = soup_1.find(class_ = 'block_info')
        info_2 = soup_1.find_all('div', class_ = 'wrapper_movies_scores_score')
        info_3 = soup_1.find_all('div', class_ = 'block_table')
        grade = [i.text.strip() for i in info_2]
        all_data = {'Название': info[0][5:]}
        grades = ['Оценка Film.ru', 'Оценка IMDb', 'Оценка пользователей']
        for i in range(len(grade)):
            if i==3:
                break
            else:
                all_data[grades[i]] = grade[i].split('\n')[0]
        titles = ['Год', 'Жанр', 'Страна']
        for i in range(len(info_1('a'))):
            if i==3:
                break
            else:
                all_data[titles[i]] = info_1('a')[i].text
        for i in range(len(info_3)):
            all_data[info_3[i].text.strip().split('\n')[0]] = info_3[i].text.strip().replace("  ", "").split('\n')[-1]
        final.append(all_data)
    return final

In [64]:
def create_file(results):
    df = pd.DataFrame(results)
    with open('films.tsv','w') as f:
        df.to_csv(f, index=False) 


In [ ]:
films = parse_web()
df = pd.DataFrame(films)
df

In [48]:
create_file(films)